In [1]:
import pandas as pd

import report_util
import tables

times = [pd.to_timedelta(5, 'm'), pd.to_timedelta(3, 'h')]
defects = tables.times_to_detected(report_util.read_timedelta_csv('../data/detections.csv'), times)
defects.head()

,campaign_id,fuzzer,subject,defect,time,detected
0,10,Zest,Bcel,B0,0 days 00:05:00,False
1,10,Zest,Bcel,B1,0 days 00:05:00,False
2,100,Zeugma-1PT,Rhino,R0,0 days 00:05:00,False
3,100,Zeugma-1PT,Rhino,R1,0 days 00:05:00,False
4,100,Zeugma-1PT,Rhino,R2,0 days 00:05:00,False


In [2]:
coverage = report_util.read_timedelta_csv('../data/coverage.csv')
coverage = coverage[coverage['time'].isin(times)]
coverage.head()

,index,time,covered_branches,subject,campaign_id,fuzzer
28,28,0 days 00:05:00,579.0,Maven,135,RLCheck
1000,1000,0 days 03:00:00,663.0,Maven,135,RLCheck
1029,28,0 days 00:05:00,287.0,Tomcat,160,Zest
2001,1000,0 days 03:00:00,343.0,Tomcat,160,Zest
2030,28,0 days 00:05:00,3152.0,Rhino,871,Zeugma-2PT


In [3]:
heritability = pd.read_csv('../data/heritability.csv')
heritability.head()

,subject,crossover_operator,inheritance_rate,hybrid
0,Rhino,Two Point,0.075963,True
1,Rhino,One Point,0.077003,True
2,Rhino,Linked,0.424558,False
3,Rhino,Two Point,0.226203,True
4,Rhino,One Point,0.299386,True


In [4]:
from report_util import get_stat_functions, compute_sig_level, compute_bucket

sig_colors = {True: 'lightblue', False: 'coolgrey'}
effect_colors = ['warmgrey', 'lightred', 'red', 'darkred']


def compute_pairwise(data, x, y):
    f1, f2, bounds2, _ = get_stat_functions(data, y)
    unique_x = sorted(data[x].unique())
    sig_level = compute_sig_level(unique_x)
    rows = []
    for r, x1 in enumerate(unique_x):
        values1 = data[data[x] == x1][y]
        row = []
        for c, x2 in enumerate(unique_x):
            values2 = data[data[x] == x2][y]
            if r > c:
                p = f1(values1, values2)
                text = f'{p:.2E}'.replace('E', r'\\E')
                color = sig_colors[p < sig_level]
            elif r < c:
                e = f2(values1, values2)
                text = f'{e:.3f}'
                color = effect_colors[compute_bucket(e, bounds2)]
            else:
                text = x1
                color = 'white'
            node = f'\\node[fill={color}] {{{text}}};'
            row.append(node)
        rows.append('&'.join(row))
    return '\\\\\n'.join(rows)

In [5]:

TEMPLATE = r"""\matrix($0)[matrix of nodes$3]{%
    $1\\%
};
\node[below =-2mm of $0, draw=none] {\textbf{$2}};"""
subjects = sorted(heritability['subject'].unique())
prev = None
for subject in subjects:
    selected = report_util.select(heritability, subject=subject)
    t = TEMPLATE.replace("$0", f'{subject.lower()}-hy') \
        .replace("$1", compute_pairwise(selected, 'crossover_operator', 'hybrid')) \
        .replace('$2', subject.title()) \
        .replace("$3", f",right=-1mm of  {prev}" if prev is not None else '')
    prev = f'{subject.lower()}-hy'
    print(t)

\matrix(ant-hy)[matrix of nodes]{%
    \node[fill=white] {Linked};&\node[fill=red] {1.506};&\node[fill=lightred] {1.314};\\
\node[fill=lightblue] {6.14\\E-06};&\node[fill=white] {One Point};&\node[fill=warmgrey] {0.873};\\
\node[fill=lightblue] {2.68\\E-03};&\node[fill=coolgrey] {1.40\\E-01};&\node[fill=white] {Two Point};\\%
};
\node[below =-2mm of ant-hy, draw=none] {\textbf{Ant}};
\matrix(bcel-hy)[matrix of nodes,right=-1mm of  ant-hy]{%
    \node[fill=white] {Linked};&\node[fill=warmgrey] {0.203};&\node[fill=warmgrey] {0.127};\\
\node[fill=lightblue] {3.21\\E-65};&\node[fill=white] {One Point};&\node[fill=warmgrey] {0.627};\\
\node[fill=lightblue] {4.74\\E-103};&\node[fill=lightblue] {2.88\\E-06};&\node[fill=white] {Two Point};\\%
};
\node[below =-2mm of bcel-hy, draw=none] {\textbf{Bcel}};
\matrix(closure-hy)[matrix of nodes,right=-1mm of  bcel-hy]{%
    \node[fill=white] {Linked};&\node[fill=lightred] {1.475};&\node[fill=warmgrey] {1.163};\\
\node[fill=lightblue] {9.10\\E-05};&\n

In [6]:
def fix_names(text):
    return text.replace("Zeugma-1PT", r"\sysnameOnePT") \
        .replace("Zeugma-2PT", r"\sysnameTwoPT") \
        .replace("Zeugma-Link", r"\sysnameLinked") \
        .replace("Zeugma-X", r"\sysnameNone") \
        .replace("RLCheck", r"\rlcheck") \
        .replace("Zest", r"\zest") \
        .replace("BeDiv-Simple", r"\bedivfuzzSimple") \
        .replace("BeDiv-Struct", r"\bedivfuzzStructure")


for time in times:
    prev = None
    for subject in subjects:
        selected = report_util.select(coverage, time=time, subject=subject)
        name = f'{subject.lower()}-{report_util.format_time_delta(time)}'
        t = TEMPLATE.replace("$0", name) \
            .replace("$1", compute_pairwise(selected, 'fuzzer', 'covered_branches')) \
            .replace('$2', f'{subject.title()} at {report_util.format_time_delta(time)}') \
            .replace("$3", f",right=-1mm of  {prev}" if prev is not None else '')
        prev = name
        print(fix_names(t))

\matrix(ant-5M)[matrix of nodes]{%
    \node[fill=white] {\bedivfuzzSimple};&\node[fill=darkred] {0.718};&\node[fill=lightred] {0.627};&\node[fill=darkred] {0.909};&\node[fill=darkred] {0.927};&\node[fill=darkred] {0.914};&\node[fill=darkred] {0.965};&\node[fill=darkred] {0.949};\\
\node[fill=coolgrey] {1.93\\E-02};&\node[fill=white] {\bedivfuzzStructure};&\node[fill=darkred] {0.900};&\node[fill=darkred] {0.858};&\node[fill=darkred] {0.863};&\node[fill=darkred] {0.869};&\node[fill=darkred] {0.907};&\node[fill=darkred] {0.896};\\
\node[fill=coolgrey] {1.69\\E-01};&\node[fill=lightblue] {1.43\\E-05};&\node[fill=white] {\rlcheck};&\node[fill=darkred] {1.000};&\node[fill=darkred] {1.000};&\node[fill=darkred] {1.000};&\node[fill=darkred] {1.000};&\node[fill=darkred] {1.000};\\
\node[fill=lightblue] {1.02\\E-05};&\node[fill=lightblue] {1.14\\E-04};&\node[fill=lightblue] {5.60\\E-08};&\node[fill=white] {\zest};&\node[fill=warmgrey] {0.559};&\node[fill=warmgrey] {0.550};&\node[fill=darkred] {0